In [1]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run, ir_datasets
from tira.rest_api_client import Client
import pyterrier as pt


In [2]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
input_dataset = 'ir-lab-sose-2024/ir-acl-anthology-20240504-training'
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', input_dataset)
# Get the topics
topics = pt_dataset.get_topics('query')

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
import gzip
import json
import pandas as pd


def create_transformer_from_json(json_file):
    # Load JSON data into DataFrame
    data = []
    with gzip.open(json_file, 'rt', encoding='utf-8') as file:
        for line in file:
            record = json.loads(line.strip())
            data.append(record)

    df = pd.DataFrame(data)
    df.rename(columns={'query_id': 'qid'}, inplace=True)

    # Create a transformer from the DataFrame
    transformer = pt.Transformer.from_df(df, uniform=True)
    return transformer

cot_prf_path = "llm-qe/cot_prf-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
cot_path = "llm-qe/cot-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2d_fs_msmarco_path = "llm-qe/q2d_fs_msmarco-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2d_fs_path =  "llm-qe/q2d_fs-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2d_prf_path = "llm-qe/q2d_prf-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2d_zs_path = "llm-qe/q2d_zs-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2e_fs_path = "llm-qe/q2e_fs-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2e_prf_path = "llm-qe/q2e_prf-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2e_zs_path = "llm-qe/q2e_zs-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"

# Create a transformer from the JSON file
cot_prf = create_transformer_from_json(cot_prf_path)
cot = create_transformer_from_json(cot_path)
q2d_fs_msmarco = create_transformer_from_json(q2d_fs_msmarco_path)
q2d_fs = create_transformer_from_json(q2d_fs_path)
q2d_prf = create_transformer_from_json(q2d_prf_path)
q2d_zs = create_transformer_from_json(q2d_zs_path)
q2e_fs = create_transformer_from_json(q2e_fs_path)
q2e_prf = create_transformer_from_json(q2e_prf_path)
q2e_zs = create_transformer_from_json(q2e_zs_path)



In [52]:
from nltk.corpus import stopwords
import re

# Ensure NLTK stopwords are available
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
import re

def preprocess_query(text):
    # Remove any unwanted characters (e.g., punctuation)
    text = re.sub(r'[^\w\s]', '', text)
    
    # Split the text into words and remove stopwords
    words = text.split()
    words = [word.lower() for word in words if word.lower() not in stop_words]
    
    # Rejoin the text
    return ' '.join(words)
    

def preprocess_llm_expansion(text):

    # Remove structures like "To address the query \"...\"" or "The term \"...\""
    text = re.sub(r'^[^"]*(query|term)[^"]*\"[^\"]*\"', '', text, flags=re.IGNORECASE).strip()
    
    # Remove text starting with patterns like "Rationale", with various symbols and spaces
    text = re.sub(r'^\s*[\*\#\n]*\s*Rationale[:\n\s]*', '', text).strip()

    # Remove the part starting with "Keywords:" and any leading/trailing whitespace
    text = re.sub(r'^\s*Keywords:\s*', '', text).strip()

    # Remove the part starting with a number and a dot (e.g., "1. ")
    text = re.sub(r'\b\d+\.\s*', '', text).strip()

    # Remove newline characters
    text = text.replace('\n', ' ')
    
    # Remove any other unwanted characters (e.g., punctuation)
    text = re.sub(r'[^\w\s]', '', text)
    
    # Split the text into words and remove stopwords
    words = text.split()
    words = [word.lower() for word in words if word.lower() not in stop_words]
    
    # Rejoin the text
    return ' '.join(words)


In [46]:
tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

def pt_tokenize(text):
    return ' '.join(tokeniser.getTokens(text))

def expand_query(topic):
  # Preprocess the llm_expansion text
  #preprocessed_query = preprocess_query(topic['query'])
  #preprocessed_expansion = preprocess_llm_expansion(topic['llm_expansion'])

  # Construct the expanded query
  #expanded_query = ' '.join([preprocessed_query] * 20 + [preprocessed_expansion])

  # Construct the expanded query
  expanded_query = ' '.join([topic['query']] * 25 + [topic['llm_expansion']])

 # Apply the tokenization
  return pt_tokenize(expanded_query)

# we wrap this into an pyterrier transformer
# Documentation: https://pyterrier.readthedocs.io/en/latest/apply.html
pt_expand_query = pt.apply.query(expand_query)

In [47]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25_bo1 = bm25 >> pt.rewrite.Bo1QueryExpansion(index) >> bm25
bm25_rm3 = bm25 >> pt.rewrite.RM3(index) >> bm25
bm25_kl = bm25 >> pt.rewrite.KLQueryExpansion(index) >> bm25

In [48]:
pipeline_q2d_zs = (q2d_zs >> pt_expand_query) >> bm25
pipeline_q2d_fs = (q2d_fs >> pt_expand_query) >> bm25
pipeline_q2d_fs_msmarco = (q2d_fs_msmarco >> pt_expand_query) >> bm25
pipeline_q2d_prf = (q2d_prf >> pt_expand_query) >> bm25

pipeline_q2e_zs = (q2e_zs >> pt_expand_query) >> bm25
pipeline_q2e_fs = (q2e_fs >> pt_expand_query) >> bm25
pipeline_q2e_prf = (q2e_prf >> pt_expand_query) >> bm25

pipeline_cot = (cot >> pt_expand_query) >> bm25
pipeline_cot_prf = (cot_prf >> pt_expand_query) >> bm25

In [61]:
print("result with query*1, expansion*1")

pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*1, expansion*1


pt.Experiment:   0%|          | 0/13 [00:00<?, ?system/s]

pt.Experiment:  15%|█▌        | 2/13 [00:04<00:28,  2.62s/system]

13:59:58.765 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
13:59:58.908 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:58<00:00,  4.47s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.334285,0.557046,0.558333,0.824720
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.233468,0.500983,0.422583,0.716882
5,BM25+q2d_fs(training data),0.249561,0.536821,0.425362,0.737549
6,BM25+q2d_fs(msmarco),0.226902,0.451403,0.430762,0.717256
7,BM25+q2d_prf,0.295683,0.576255,0.475840,0.759188
8,BM25+q2e_zs,0.203412,0.386732,0.379809,0.691984
9,BM25+q2e_fs,0.236367,0.454178,0.454391,0.791643


In [57]:
print("result with query*5, expansion *1")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*5, expansion *1


pt.Experiment:   0%|          | 0/13 [00:00<?, ?system/s]

pt.Experiment:  15%|█▌        | 2/13 [00:05<00:35,  3.19s/system]

13:57:54.164 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
13:57:54.350 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:59<00:00,  4.57s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.334285,0.557046,0.558333,0.824720
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.298996,0.567174,0.517039,0.803946
5,BM25+q2d_fs(training data),0.327689,0.606379,0.523259,0.816709
6,BM25+q2d_fs(msmarco),0.319042,0.520409,0.545020,0.831599
7,BM25+q2d_prf,0.337850,0.607218,0.536623,0.835209
8,BM25+q2e_zs,0.330336,0.551438,0.530583,0.825605
9,BM25+q2e_fs,0.350054,0.553105,0.589487,0.854271


In [64]:
print("result with query*10, expansion *1")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*10, expansion *1


pt.Experiment:  15%|█▌        | 2/13 [00:04<00:29,  2.65s/system]

14:01:21.111 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
14:01:21.267 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:57<00:00,  4.41s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.334285,0.557046,0.558333,0.824720
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.351865,0.582197,0.570003,0.840642
5,BM25+q2d_fs(training data),0.369316,0.632794,0.594531,0.856184
6,BM25+q2d_fs(msmarco),0.370676,0.575057,0.594361,0.861028
7,BM25+q2d_prf,0.367564,0.625277,0.582286,0.847897
8,BM25+q2e_zs,0.381493,0.593133,0.576018,0.843157
9,BM25+q2e_fs,0.368586,0.586201,0.603162,0.849416


In [7]:
print("result with query*15, expansion *1")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*15, expansion *1


pt.Experiment:  15%|█▌        | 2/13 [00:08<00:45,  4.18s/system]

20:24:50.387 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
20:24:50.686 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [01:02<00:00,  4.79s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.333132,0.556066,0.558675,0.825062
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.372744,0.592368,0.595238,0.854988
5,BM25+q2d_fs(training data),0.398545,0.643204,0.622219,0.865024
6,BM25+q2d_fs(msmarco),0.384317,0.576978,0.608771,0.863855
7,BM25+q2d_prf,0.386681,0.640153,0.592344,0.849028
8,BM25+q2e_zs,0.406101,0.625979,0.593796,0.851111
9,BM25+q2e_fs,0.374360,0.595947,0.601964,0.845695


In [31]:
print("result with query*15, expansion *1, after preprocessing")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*15, expansion *1, after preprocessing


pt.Experiment:  15%|█▌        | 2/13 [00:04<00:28,  2.63s/system]

21:26:01.190 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
21:26:01.342 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:54<00:00,  4.21s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.333132,0.556066,0.558675,0.825062
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.365256,0.592657,0.580239,0.848949
5,BM25+q2d_fs(training data),0.384853,0.640486,0.585933,0.854509
6,BM25+q2d_fs(msmarco),0.376239,0.584377,0.594606,0.853317
7,BM25+q2d_prf,0.384693,0.660809,0.574061,0.840667
8,BM25+q2e_zs,0.389724,0.597116,0.574952,0.834870
9,BM25+q2e_fs,0.363790,0.614915,0.588320,0.832709


In [35]:
print("result with query*10, expansion *1, after preprocessing")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*10, expansion *1, after preprocessing


pt.Experiment:  15%|█▌        | 2/13 [00:05<00:29,  2.73s/system]

21:32:45.948 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
21:32:46.098 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:53<00:00,  4.09s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.333132,0.556066,0.558675,0.825062
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.336294,0.570919,0.554579,0.835312
5,BM25+q2d_fs(training data),0.357766,0.627894,0.564326,0.845723
6,BM25+q2d_fs(msmarco),0.364554,0.568302,0.577707,0.850297
7,BM25+q2d_prf,0.367188,0.655726,0.568191,0.839849
8,BM25+q2e_zs,0.367191,0.588589,0.566226,0.831439
9,BM25+q2e_fs,0.364375,0.592286,0.588878,0.836302


In [39]:
print("result with query*5, expansion *1, after preprocessing")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*5, expansion *1, after preprocessing


pt.Experiment:  15%|█▌        | 2/13 [00:04<00:28,  2.60s/system]

22:01:27.985 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
22:01:28.139 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:52<00:00,  4.04s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.333132,0.556066,0.558675,0.825062
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.294992,0.577681,0.511899,0.797245
5,BM25+q2d_fs(training data),0.315823,0.580310,0.507103,0.806618
6,BM25+q2d_fs(msmarco),0.308646,0.535534,0.531018,0.825799
7,BM25+q2d_prf,0.334545,0.638447,0.526926,0.829378
8,BM25+q2e_zs,0.343423,0.577735,0.534169,0.826003
9,BM25+q2e_fs,0.344075,0.576730,0.583649,0.843095


In [42]:
print("result with query*20, expansion *1, after preprocessing")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*20, expansion *1, after preprocessing


pt.Experiment:  15%|█▌        | 2/13 [00:04<00:28,  2.57s/system]

22:08:18.074 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
22:08:18.329 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:53<00:00,  4.08s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.334285,0.557046,0.558333,0.824720
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.371936,0.588018,0.595006,0.852749
5,BM25+q2d_fs(training data),0.398131,0.653465,0.601081,0.857432
6,BM25+q2d_fs(msmarco),0.382590,0.598253,0.597257,0.847372
7,BM25+q2d_prf,0.388763,0.652575,0.585300,0.839835
8,BM25+q2e_zs,0.386689,0.606977,0.576502,0.836163
9,BM25+q2e_fs,0.366553,0.607978,0.589450,0.829349


In [45]:
print("result with query*20, expansion *1, w/o preprocessing")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*20, expansion *1, w/o preprocessing


pt.Experiment:  15%|█▌        | 2/13 [00:04<00:28,  2.62s/system]

22:14:56.098 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
22:14:56.237 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:57<00:00,  4.43s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.334285,0.557046,0.558333,0.824720
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.380773,0.599548,0.604841,0.862501
5,BM25+q2d_fs(training data),0.414328,0.657220,0.623204,0.865890
6,BM25+q2d_fs(msmarco),0.390398,0.585181,0.621013,0.859740
7,BM25+q2d_prf,0.396303,0.641694,0.602382,0.852781
8,BM25+q2e_zs,0.406985,0.653156,0.597851,0.850967
9,BM25+q2e_fs,0.375342,0.598034,0.606304,0.842479


In [49]:
print("result with query*25, expansion *1, w/o preprocessing")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*25, expansion *1, w/o preprocessing


pt.Experiment:  15%|█▌        | 2/13 [00:04<00:29,  2.66s/system]

22:19:49.446 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
22:19:49.590 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:57<00:00,  4.41s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.333132,0.556066,0.558675,0.825062
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.387856,0.593766,0.611979,0.858981
5,BM25+q2d_fs(training data),0.418457,0.653673,0.632858,0.867481
6,BM25+q2d_fs(msmarco),0.396637,0.611011,0.618855,0.861890
7,BM25+q2d_prf,0.400763,0.641696,0.605106,0.846131
8,BM25+q2e_zs,0.401505,0.634558,0.596728,0.849841
9,BM25+q2e_fs,0.378154,0.602714,0.607170,0.839946
